# 构建文本嵌入模型

嵌入：文本数据转为数值表示的过程

嵌入模型：对输入进行嵌入的 LLM 模型

目的：尽可能准确的将文本数据表示为嵌入向量（表示包含语义、情感等基于不同目的）

## 对比学习

训练和微调文本嵌入的一种主要技术

基本理念：向模型输入相似和不相似的文档对作为示例。 对比解释是通过“为什么是 P 而不是 Q”来理解“为什么是 P”

## SBERT

bi-encoder 或 sentences-BERT 双编码器架构

是 sentence-transformers 使用的训练的一种孪生架构

训练过程：
1. 文本分别输入两个完全相同共享权重的 BERT 模型
2. 输出层做平均池化生成嵌入向量
3. 文本的嵌入和之间的差向量拼接
4. 使用softmax 分类器对嵌入向量进行优化

## 构建嵌入模型

### 生成对比样本

NLI（自然语言推理）数据集

GLUE 基准评估和分析模型性能

### 训练

1. 选择 BERT 基座模型
2. 定义损失函数


In [1]:
from datasets import load_dataset

train_dataset = load_dataset('glue', 'mnli', split='train').select(range(50_000))
train_dataset = train_dataset.remove_columns('idx')

train_dataset[0]

README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/52.2M [00:00<?, ?B/s]

(…)alidation_matched-00000-of-00001.parquet:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

(…)dation_mismatched-00000-of-00001.parquet:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

test_matched-00000-of-00001.parquet:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

...

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Generating test_matched split:   0%|          | 0/9796 [00:00<?, ? examples/s]

Generating test_mismatched split:   0%|          | 0/9847 [00:00<?, ? examples/s]

{'premise': 'Conceptually cream skimming has two basic dimensions - product and geography.',
 'hypothesis': 'Product and geography are what make cream skimming work. ',
 'label': 1}

In [3]:
from sentence_transformers import SentenceTransformer, losses
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

# base model
embedding_model = SentenceTransformer('bert-base-uncased')
# critetion / loss function
train_loss = losses.SoftmaxLoss(
    model=embedding_model,
    sentence_embedding_dimension=embedding_model.get_sentence_embedding_dimension(),
    num_labels=3,
)
# evaluation
val_sts = load_dataset('glue', 'stsb', split='validation')
evaluator = EmbeddingSimilarityEvaluator(
    sentences1=val_sts['sentence1'],
    sentences2=val_sts['sentence2'],
    scores=[score/5 for score in val_sts['label']],
    main_similarity='cosine'
)


No sentence-transformers model found with name bert-base-uncased. Creating a new one with mean pooling.


train-00000-of-00001.parquet:   0%|          | 0.00/502k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/151k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/114k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5749 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1379 [00:00<?, ? examples/s]

In [8]:
from sentence_transformers.training_args import SentenceTransformerTrainingArguments
from sentence_transformers.trainer import SentenceTransformerTrainer
import torch

# 禁用mps， fp16 加速不能使用 mps
# torch.cuda.is_available = lambda: False
torch.backends.mps.is_available = lambda: False

args = SentenceTransformerTrainingArguments(
    output_dir='base_embedding_model',
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=100,
    fp16=True,
    eval_steps=100,
    logging_steps=100,
)

trainer = SentenceTransformerTrainer(
    model=embedding_model,
    args=args,
    train_dataset=train_dataset,
    evaluator=evaluator,
    loss=train_loss,
)

trainer.train()

Column 'hypothesis' is at index 1, whereas a column with this name is usually expected at index 0. Note that the column order can be important for some losses, e.g. MultipleNegativesRankingLoss will always consider the first column as the anchor and the second as the positive, regardless of the dataset column names. Consider renaming the columns to match the expected order, e.g.:
dataset = dataset.select_columns(['hypothesis', 'entailment', 'contradiction'])


Step,Training Loss
100,1.035600
200,0.886400


KeyboardInterrupt: 

In [9]:
%pip freeze > requirements.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.
